In [6]:
import random
import speech_recognition as sr
import tkinter as tk

# Lists of random words to construct sentences
subjects = ["the cat", "a dog", "a bird", "the man", "a woman", "the child", "an elephant", "the teacher", "the student", "a programmer"]
verbs = ["eats", "jumps over", "runs", "walks", "sits on", "stands by", "talks to", "reads", "writes", "plays with"]
objects = ["a book", "the grass", "a tree", "a computer", "a ball", "the food", "a chair", "a game", "the phone", "a car"]
additional_phrases = ["quickly", "happily", "sadly", "angrily", "excitedly", "silently", "loudly", "carefully", "eagerly", "curiously"]

def generate_random_sentence(num_words=10):
    sentence = []
    for _ in range(num_words // 4):  # Each sentence will have approximately subject-verb-object-adverb structure
        subject = random.choice(subjects)
        verb = random.choice(verbs)
        obj = random.choice(objects)
        additional = random.choice(additional_phrases)
        sentence.extend([subject, verb, obj, additional])
    return ' '.join(sentence[:num_words]).lower()

def compare_sentences(random_sentence, user_sentence):
    random_words = random_sentence.split()
    user_words = user_sentence.split()
    min_length = min(len(random_words), len(user_words))
    
    correct_indices = []
    for i in range(min_length):
        if random_words[i] == user_words[i]:
            correct_indices.append(i)
    
    return correct_indices

def recognize_speech_from_mic(recognizer, microphone, timeout=5, retry_limit=3):
    attempts = 0
    while attempts < retry_limit:
        with microphone as source:
            recognizer.adjust_for_ambient_noise(source)
            print("Listening...")
            audio = recognizer.listen(source, timeout=timeout)

        response = {
            "success": True,
            "error": None,
            "transcription": None
        }

        try:
            response["transcription"] = recognizer.recognize_google(audio).lower()
            return response
        except sr.RequestError:
            response["success"] = False
            response["error"] = "API unavailable"
        except sr.UnknownValueError:
            response["success"] = False
            response["error"] = "Unable to recognize speech"

        attempts += 1
        print(f"Recognition attempt {attempts}/{retry_limit} failed.")

    return response

class SentenceGameApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sentence Game")

        self.random_sentence = generate_random_sentence(num_words=10)

        self.label_sentence = tk.Label(self.root, text=self.random_sentence)
        self.label_sentence.pack(pady=20)

        self.label_feedback = tk.Label(self.root, text="Repeat the sentence:", font=("Arial", 12))
        self.label_feedback.pack(pady=10)

        self.entry_sentence = tk.Entry(self.root, font=("Arial", 14))
        self.entry_sentence.pack(pady=10)

        self.button_listen = tk.Button(self.root, text="Listen", command=self.listen_repeat)
        self.button_listen.pack(pady=10)

        self.label_result = tk.Label(self.root, text="", font=("Arial", 12))
        self.label_result.pack(pady=10)

    def listen_repeat(self):
        recognizer = sr.Recognizer()
        microphone = sr.Microphone()

        response = recognize_speech_from_mic(recognizer, microphone, timeout=5, retry_limit=3)

        if response["success"]:
            user_sentence = response["transcription"]
            self.entry_sentence.delete(0, tk.END)
            self.entry_sentence.insert(0, user_sentence)

            correct_indices = compare_sentences(self.random_sentence, user_sentence)
            self.update_feedback(correct_indices)

            if len(correct_indices) == len(self.random_sentence.split()):
                self.label_result.config(text="You did it! The sentence matches.")
            else:
                self.label_result.config(text="The sentence does not match.")

        else:
            self.label_result.config(text="I didn't catch that. Please try again.")

    def update_feedback(self, correct_indices):
        sentence_words = self.random_sentence.split()
        feedback_text = ""
        for index, word in enumerate(sentence_words):
            if index in correct_indices:
                feedback_text += word + " "
            else:
                feedback_text += word + " "

        self.label_feedback.config(text=feedback_text.strip())

if __name__ == "__main__":
    root = tk.Tk()
    app = SentenceGameApp(root)
    root.mainloop()
